In [1]:
import requests #пакет для парсинга, т.е. для извлечения информации с сайта. мы будем парсить сайт, имеющий api
import json #пакет для работы с данными в формате json
import re #пакет для работы с регулярными выражениями
import pandas as pd #пакет для очистки и первичного анализа данных
import math #пакет для выполнения математических операций
import numpy as np #продвинутый пакет для выполнения математических операций и операций с массивами

In [2]:
url = 'https://api.openalex.org/works?filter=host_venue.id:S58059009' 
rj = requests.get(url).json()
rj

{'meta': {'count': 1907,
  'db_response_time_ms': 551,
  'page': 1,
  'per_page': 25},
 'results': [{'id': 'https://openalex.org/W3093161410',
   'doi': 'https://doi.org/10.31857/s013216250009460-9',
   'title': 'All-Russian National Identity: Searching for Definition and Distribution Dynamics',
   'display_name': 'All-Russian National Identity: Searching for Definition and Distribution Dynamics',
   'publication_year': 2020,
   'publication_date': '2020-08-30',
   'ids': {'openalex': 'https://openalex.org/W3093161410',
    'doi': 'https://doi.org/10.31857/s013216250009460-9',
    'mag': '3093161410'},
   'primary_location': {'is_oa': False,
    'landing_page_url': 'https://doi.org/10.31857/s013216250009460-9',
    'pdf_url': None,
    'source': {'id': 'https://openalex.org/S58059009',
     'display_name': 'Sotsiologicheskie Issledovaniya',
     'issn_l': '0132-1625',
     'issn': ['0132-1625'],
     'host_organization': 'https://openalex.org/P4310312666',
     'host_organization_name'

In [3]:
print('Number of results in current rj variable:', len(rj['results']))

print('Total number of results:', rj['meta']['count'])
n_pages = math.ceil(rj['meta']['count']/rj['meta']['per_page'])
print ('Total number of pages:', n_pages)

Number of results in current rj variable: 25
Total number of results: 1907
Total number of pages: 77


In [4]:
data = rj["results"] # сохраняем поле results c уже имеющейся первой страницы в переменную data

# Начиная со второй страницы и далее до последней страницы, проходим по всем страницам 
# и добавляем содержимое поля results с каждой страницы в переменную data

for page in range(2, n_pages+1):
    print(page, end = " ")
    rj_new = requests.get(url + f'&page={page}').json()
    data.extend(rj_new['results'])

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 

In [5]:

df = pd.DataFrame(columns = ['doi', 'title', 'year', 'cited_by', 'N_authors']) 

for index, item in enumerate(data): #перебираем результаты и берем нужные поля
    df.at[index,'doi'] = item['doi'] #вносим результат в нужную колонку
    df.at[index,'title'] = item['title']
    df.at[index,'year'] = item['publication_year']
    df.at[index,'source'] = item['host_venue']['display_name']
    df.at[index,'cited_by'] = item['cited_by_count']
    df.at[index,'N_authors'] = len(item['authorships'])
    # дополнительно создаем отдельные колонки id_№ и name_№ для каждого автора
    for tag in range(len(item['authorships'])):
        num = str(tag + 1)
        df.at[index,'id_'+num] = item['authorships'][tag]['author']['id']
        df.at[index,'name_'+num] = item['authorships'][tag]['author']['display_name']  
        
df.info() #просмотр описания сохраненной таблицы

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1907 entries, 0 to 1906
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   doi        1229 non-null   object
 1   title      1907 non-null   object
 2   year       1907 non-null   object
 3   cited_by   1907 non-null   object
 4   N_authors  1907 non-null   object
 5   source     1907 non-null   object
 6   id_1       1725 non-null   object
 7   name_1     1749 non-null   object
 8   id_2       653 non-null    object
 9   name_2     643 non-null    object
 10  id_3       243 non-null    object
 11  name_3     244 non-null    object
 12  id_4       61 non-null     object
 13  name_4     56 non-null     object
 14  id_5       16 non-null     object
 15  name_5     13 non-null     object
 16  id_6       8 non-null      object
 17  name_6     7 non-null      object
 18  id_7       6 non-null      object
 19  name_7     6 non-null      object
 20  id_8       4 non-null      obj

In [6]:
df.head()

,doi,title,year,cited_by,N_authors,source,id_1,name_1,id_2,name_2,...,id_8,name_8,id_9,name_9,id_10,name_10,id_11,name_11,id_12,name_12
0,https://doi.org/10.31857/s013216250009460-9,All-Russian National Identity: Searching for D...,2020,12,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2809957441,Леокадия Михайловна Дробижева,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,Manageability: Sociological theoretical analys...,2007,11,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A3126989184,M. V. Rubtsova,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://doi.org/10.7868/s0132162518030029,МИЛЛЕНИАЛЫ НА ФОНЕ ПРЕДШЕСТВУЮЩИХ ПОКОЛЕНИЙ: Э...,2018,11,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A4305835872,В.В. Радаев,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://doi.org/10.31857/s013216250008325-0,From «Smart Regulation» to «Smart Management»:...,2020,9,2,Sotsiologicheskie Issledovaniya,https://openalex.org/A4348437703,Alexander N. Tikhonov,https://openalex.org/A4357908561,Vladimir Y. Bogdanov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://doi.org/10.31857/s013216250002784-5,Dynamics of the Social Attitudes of the Russia...,2018,8,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A3075109796,V. O. Petukhov,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,doi,title,year,cited_by,N_authors,source,id_1,name_1,id_2,name_2,...,id_8,name_8,id_9,name_9,id_10,name_10,id_11,name_11,id_12,name_12
1902,None,Истина и политический контекст в научных практ...,2005,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A4334462866,Юрий Львович Качанов,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903,None,Уклонение от уплаты налогов (опыт анализа соци...,2007,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2479278786,Санина Анна Георгиевна,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904,None,Субъективность в качественном исследовании: но...,2007,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2587829985,Старовойтенко Елена Борисовна,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905,None,Особенности построения ценовых стратегий вузам...,2009,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2055379307,Олександра Олегівна Логунова,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906,None,И снова о женском предпринимательстве,2008,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2128162305,Барсукова Светлана Юрьевна,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
print(df['year'].value_counts(dropna = False, sort = False))

2020    245
2007     13
2018    318
2019    272
1988      3
2017    158
2021    273
1978      1
2022    168
2009     24
2008     21
2010     21
1982      4
1992     14
2005     30
1997      6
2016     45
2012     26
2013     34
2011     30
2015     40
2002      7
2004      9
2006     14
1999      7
1993      4
2001      3
2003      7
1990      6
1996     12
1994     11
1984      4
1991      5
1981      7
1987      2
1998      4
1969      2
1986      3
1983      1
2014     39
1979      1
1989      5
1985      2
2000      6
Name: year, dtype: int64


In [12]:
df = df[df['name_1'].notna()] # оставим только те публикации, у которых есть хотя бы один автор 
df = df[df['title'].notna()] # оставим только те публикации, у которых есть есть название
df = df.loc[(df['year']>=2019) & (df['year']<=2020)] # ограничим данные статьями, опубликованными в 2019-2020 гг.

# В Ваших журналах могут быть другие проблемы, действуйте по обстоятельствам
# временной интервал выбирайте также исходя из Ваших обстоятельств, но не позднее 2021 г. 
# и не менее 200 статей в сумме за весь период

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516 entries, 0 to 1689
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   doi        448 non-null    object
 1   title      516 non-null    object
 2   year       516 non-null    object
 3   cited_by   516 non-null    object
 4   N_authors  516 non-null    object
 5   source     516 non-null    object
 6   id_1       513 non-null    object
 7   name_1     516 non-null    object
 8   id_2       208 non-null    object
 9   name_2     208 non-null    object
 10  id_3       82 non-null     object
 11  name_3     82 non-null     object
 12  id_4       24 non-null     object
 13  name_4     24 non-null     object
 14  id_5       3 non-null      object
 15  name_5     3 non-null      object
 16  id_6       1 non-null      object
 17  name_6     1 non-null      object
 18  id_7       1 non-null      object
 19  name_7     1 non-null      object
 20  id_8       1 non-null      obje

In [13]:
df.head()

,doi,title,year,cited_by,N_authors,source,id_1,name_1,id_2,name_2,...,id_8,name_8,id_9,name_9,id_10,name_10,id_11,name_11,id_12,name_12
0,https://doi.org/10.31857/s013216250009460-9,All-Russian National Identity: Searching for D...,2020,12,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2809957441,Леокадия Михайловна Дробижева,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://doi.org/10.31857/s013216250008325-0,From «Smart Regulation» to «Smart Management»:...,2020,9,2,Sotsiologicheskie Issledovaniya,https://openalex.org/A4348437703,Alexander N. Tikhonov,https://openalex.org/A4357908561,Vladimir Y. Bogdanov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://doi.org/10.31857/s013216250009390-2,Digitalization and Creativity of Russian Regions,2020,8,2,Sotsiologicheskie Issledovaniya,https://openalex.org/A3102113549,Igor Lvovich Groshev,https://openalex.org/A1942750792,Alexey V. Krasnoslobodtsev,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://doi.org/10.31857/s013216250008802-5,Simulation of Educational Practices in the Sys...,2020,7,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A4330381496,Nina Anatolievna Seliverstova,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,https://doi.org/10.31857/s013216250009116-0,Russian Citizens’ Health Self-assessment Dynam...,2020,7,2,Sotsiologicheskie Issledovaniya,https://openalex.org/A2286185998,Козырева Полина Михайловна,https://openalex.org/A2577301055,Смирнов Александр Ильич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df[df['doi'].isnull()].tail()

,doi,title,year,cited_by,N_authors,source,id_1,name_1,id_2,name_2,...,id_8,name_8,id_9,name_9,id_10,name_10,id_11,name_11,id_12,name_12
1447,None,"Шубрт И. Антиномии, дилеммы и дискуссии в совр...",2020,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2955171323,А Никитенкова Мария,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1475,None,Потребительские практики: изучение покупательс...,2019,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A4344407797,Елена Дмитриевна Твердюкова,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,None,Изучение покупательского спроса в СССР (конец ...,2019,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A3011593121,Д Сушко Елена,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1530,None,Софтверизация общества: истоки и перспективы,2019,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2549143697,Е.В. Журавлева,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1540,None,Изменение характеристик социологии как науки (...,2019,0,1,Sotsiologicheskie Issledovaniya,https://openalex.org/A2559640241,Вашанов Вячеслав,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
female_end_list = ['ova', 'ina', 'eva', 'aia', 'aya', 'vna', 'ова', 'ина', 'ева', 'ая', 'вна']
male_end_list = ['ov', 'in', 'ev', 'ky', 'ii', 'yi', 'iy', 'ny', 'oy', 'oi', 'ин', 'ов', 'ев', 'ий', 'ой']

def Author_gender(Author):
    gender_list = []
    gender = ''
    auth_list = Author.split(sep = ' ')
    for auth in auth_list:
        for i in female_end_list:
            if auth.endswith(i) == True:
                gender_list.append('f')
        for i in male_end_list:
            if auth.endswith(i) == True:
                gender_list.append('m')
    if (gender_list == ['m']) | (gender_list == ['m', 'm']):
        gender = 'm'
    elif (gender_list == ['f']) | (gender_list == ['f', 'f']):
        gender = 'f'
    else:
        gender = '???'
    return gender   

In [16]:
df['N_authors'] = df['N_authors'].astype('int')
max_n_authors = df['N_authors'].max()

for x in range(1, max_n_authors + 1):
    str_x = str(x)
    # чистка данных по id и имени авторов с использованием синтаксиса регулярных выражений
    df[f'name_{str_x}'] = df[f'name_{str_x}'].str.replace('\.', '', regex=True)
    df[f'name_{str_x}'] = df[f'name_{str_x}'].str.replace('  ', ' ', regex=True)    
    df[f'name_{str_x}'] = df[f'name_{str_x}'].str.replace('^\s+', '', regex=True)
    df[f'name_{str_x}'] = df[f'name_{str_x}'].str.replace('\s+$', '', regex=True)
    df[f'id_{str_x}'] = df[f'id_{str_x}'].replace('https://openalex.org/', '', regex=True)
    # Создание переменных пола авторов
    df[f'gender_{str_x}'] = df[f'name_{str_x}'].apply(lambda x: Author_gender(x) if isinstance(x, str) else x)
    
df.tail()

,doi,title,year,cited_by,N_authors,source,id_1,name_1,id_2,name_2,...,id_12,name_12,gender_1,gender_2,gender_3,gender_4,gender_5,gender_6,gender_7,gender_8
1540,None,Изменение характеристик социологии как науки (...,2019,0,1,Sotsiologicheskie Issledovaniya,A2559640241,Вашанов Вячеслав,NaN,NaN,...,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1676,https://doi.org/10.31857/s013216250010234-0,Developing the Ideas of S.I. Golod,2020,0,1,Sotsiologicheskie Issledovaniya,A3216080333,Olga N Burmykina,NaN,NaN,...,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1681,https://doi.org/10.31857/s013216250009351-9,The Role of the 2018 FIFA World Cup in Forming...,2020,0,1,Sotsiologicheskie Issledovaniya,A2663456607,V V Kochetkov,NaN,NaN,...,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1688,https://doi.org/10.31857/s013216250009498-0,Journal «Aleumettanu-SOCIOLOGY» is 10!,2020,0,2,Sotsiologicheskie Issledovaniya,A3203242442,Serik Seidumanov,A3199471553,Zarema K Shaukenova,...,NaN,NaN,m,f,NaN,NaN,NaN,NaN,NaN,NaN
1689,https://doi.org/10.31857/s013216250009292-4,«No Need to Slay Chicken that Lays Golden Eggs...,2020,0,1,Sotsiologicheskie Issledovaniya,A2898065801,Grigory Y Kanarsh,NaN,NaN,...,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.to_excel('Socio_Issledovania.xlsx') 